In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
# from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD 
from sklearn.preprocessing import RobustScaler

# 숫자 표기법 변환
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. 데이터 셋 만들기

In [4]:
# train = pd.read_csv("/content/drive/Shared drives/빅콘테스트2020/data/data/01_제공데이터/train.csv")
test = pd.read_csv('/content/drive/Shared drives/빅콘테스트2020/real_test.csv')

In [5]:
test

,노출(분)_주문량,상품군_주문량,시_분_주문량,요일_주문량,휴일_주문량,season_주문량,시등급_주문량,season,시,분,노출횟수,노출횟수등급_주문량,분등급_주문량,판매단가등급,판매단가등급_주문량,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
0,326.28,384.96,236.56,310.82,305.48,325.72,286.35,1,6,20,1,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
1,326.28,384.96,348.36,310.82,305.48,325.72,286.35,1,6,40,2,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
2,326.28,384.96,276.45,310.82,305.48,325.72,572.83,1,7,0,3,443.94,251.58,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
3,326.28,364.46,387.39,310.82,305.48,325.72,572.83,1,7,20,1,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,0.02,0.01,...,-0.01,-0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,-0.00,0.01,-0.01,0.01,0.00,0.01,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,-0.01,0.00,0.00,0.00,-0.00,0.00
4,326.28,364.46,527.57,310.82,305.48,325.72,572.83,1,7,40,2,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,0.02,0.01,...,-0.01,-0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,-0.00,0.01,-0.01,0.01,0.00,0.01,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,-0.01,0.00,0.00,0.00,-0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2886,326.28,347.12,221.06,299.97,305.48,325.72,286.35,1,0,20,1,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,...,0.00,0.00,-0.00,-0.01,-0.01,-0.00,-0.01,-0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.01,0.01,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.01,-0.00,0.00,-0.00
2887,326.28,347.12,274.53,299.97,305.48,325.72,286.35,1,0,40,2,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,...,0.00,0.00,-0.00,-0.01,-0.01,-0.00,-0.01,-0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.01,-0.00,-0.00,-0.01,0.01,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.01,-0.00,0.00,-0.00
2888,326.28,347.12,139.00,299.97,305.48,325.72,140.26,1,1,0,3,443.94,251.58,1,601.61,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.

In [48]:
product1 = test.iloc[:3,:]
product2 = test.iloc[3:6,:]
product3 = test.iloc[6:9,:]
product4 = test.iloc[9:12,:]
product5 = test.iloc[12:15,:]
product6 = test.iloc[15:18,:]
product7 = test.iloc[18:21,:]
product8 = test.iloc[21:27,:] # 6개
product9 = test.iloc[27:28,:] # 1개
product10 = test.iloc[28:31,:]
product11 = test.iloc[31:34,:]
product12 = test.iloc[34:40,:] # 6개
product13 = test.iloc[40:43,:]
product14 = test.iloc[43:46,:]
product15 = test.iloc[46:49,:]
product16 = test.iloc[49:58,:] # 9개
product17 = test.iloc[58:67,:] # 9개
product18 = test.iloc[67:73,:] # 6개
product19 = test.iloc[73:76,:]
product20 = test.iloc[76:79,:]

In [49]:
product20

,노출(분)_주문량,상품군_주문량,시_분_주문량,요일_주문량,휴일_주문량,season_주문량,시등급_주문량,season,시,분,노출횟수,노출횟수등급_주문량,분등급_주문량,판매단가등급,판매단가등급_주문량,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
76,326.28,250.46,179.83,311.63,305.48,325.72,140.26,1,1,20,1,259.14,353.50,2,121.58,0.00,0.01,-0.00,0.00,-0.01,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.01,-0.00,-0.00,0.00,...,-0.00,-0.01,0.01,-0.00,-0.00,0.01,0.01,-0.00,0.01,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,-0.01,-0.00,-0.01,-0.00,-0.01,0.00,-0.00,-0.00,0.01,-0.00,0.00
77,155.02,250.46,209.72,311.63,305.48,325.72,140.26,1,1,40,2,259.14,353.50,2,121.58,0.00,0.01,-0.00,0.00,-0.01,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.01,-0.00,-0.00,0.00,...,-0.00,-0.01,0.01,-0.00,-0.00,0.01,0.01,-0.00,0.01,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,-0.01,-0.00,-0.01,-0.00,-0.01,0.00,-0.00,-0.00,0.01,-0.00,0.00
78,326.28,250.46,243.96,311.63,305.48,325.72,286.35,1,2,0,3,443.94,251.58,2,121.58,0.00,0.01,-0.00,0.00,-0.01,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.01,-0.00,-0.00,0.00,...,-0.00,-0.01,0.01,-0.00,-0.00,0.01,0.01,-0.00,0.01,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,-0.01,-0.00,-0.01,-0.00,-0.01,0.00,-0.00,-0.00,0.01,-0.00,0.00


In [56]:
product2

,노출(분)_주문량,상품군_주문량,시_분_주문량,요일_주문량,휴일_주문량,season_주문량,시등급_주문량,season,시,분,노출횟수,노출횟수등급_주문량,분등급_주문량,판매단가등급,판매단가등급_주문량,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
3,326.28,364.46,387.39,310.82,305.48,325.72,572.83,1,7,20,1,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,0.02,0.01,...,-0.01,-0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,-0.00,0.01,-0.01,0.01,0.00,0.01,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,-0.01,0.00,0.00,0.00,-0.00,0.00
4,326.28,364.46,527.57,310.82,305.48,325.72,572.83,1,7,40,2,259.14,353.50,1,601.61,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,0.02,0.01,...,-0.01,-0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,-0.00,0.01,-0.01,0.01,0.00,0.01,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,-0.01,0.00,0.00,0.00,-0.00,0.00
5,326.28,364.46,249.73,310.82,305.48,325.72,379.54,1,8,0,3,443.94,251.58,1,601.61,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.01,-0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.01,0.00,0.00,0.00,0.00,-0.00,0.02,0.01,...,-0.01,-0.01,-0.00,-0.00,0.00,0.01,-0.01,-0.00,0.00,0.00,-0.00,0.01,0.00,0.01,0.01,-0.01,-0.01,0.00,0.01,0.00,-0.00,-0.00,0.01,-0.01,0.01,0.00,0.01,0.01,0.01,-0.01,-0.01,0.01,0.00,-0.00,-0.01,0.00,0.00,0.00,-0.00,0.00


In [53]:
product1_total = pd.DataFrame()

In [54]:
for i in range(20):
  temp = product1.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  product1_total = product1_total.append(temp)
  product1_total = product1_total.reset_index(drop=True)

In [55]:
product1_total

,노출(분)_주문량,상품군_주문량,시_분_주문량,요일_주문량,휴일_주문량,season_주문량,시등급_주문량,season,시,분,노출횟수,노출횟수등급_주문량,분등급_주문량,판매단가등급,판매단가등급_주문량,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164
0,326.28,384.96,236.56,310.82,305.48,325.72,286.35,1,6,20,1,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
1,326.28,384.96,348.36,310.82,305.48,325.72,286.35,1,6,40,2,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
2,326.28,384.96,276.45,310.82,305.48,325.72,572.83,1,7,0,3,443.94,251.58,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
3,326.28,384.96,236.56,310.82,305.48,325.72,286.35,1,7,20,1,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
4,326.28,384.96,348.36,310.82,305.48,325.72,286.35,1,7,40,2,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
5,326.28,384.96,276.45,310.82,305.48,325.72,572.83,1,8,0,3,443.94,251.58,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
6,326.28,384.96,236.56,310.82,305.48,325.72,286.35,1,8,20,1,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
7,326.28,384.96,348.36,310.82,305.48,325.72,286.35,1,8,40,2,259.14,353.50,1,601.61,0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.01,-0.00,...,-0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00
8,326.28,384.96,276.45,310.82,305.48,325.72,572.83,1,9,0,3,443.94,25

In [6]:
scaler = RobustScaler()

scaler.fit(test)

test2 = pd.DataFrame(scaler.transform(test), columns=test.columns)

## 모델링

In [11]:
from keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D, Conv1DTranspose, concatenate, Input, add, Dropout, BatchNormalization
from tensorflow.keras import Model

def build_model(input_layer, start_neurons):
    
    conv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same", input_shape=(test.shape[1], 1))(input_layer)
    conv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = AveragePooling1D(2)(pool1)
    pool1 = Dropout(0.1)(pool1)

    conv2 = Conv1D(start_neurons * 2, 3, activation="relu", padding="same")(pool1)
    conv2 = Conv1D(start_neurons * 2, 3, activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = AveragePooling1D(2)(pool2)
    pool2 = Dropout(0.1)(pool2)

    convm = Conv1D(start_neurons * 4, 3, activation="relu", padding="same")(pool2)

    deconv2 = Conv1DTranspose(start_neurons * 1, 3, strides=2, padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv2)
    uconv2 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Dropout(0.1)(uconv2)

    deconv1 = Conv1DTranspose(start_neurons * 1, 3, strides=2, padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv1)
    uconv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.1)(uconv1)

    flat = Flatten()(uconv1)
    dense1 = Dense(256, activation='relu')(flat)
    dense2 = Dense(128, activation='relu')(dense1)
    output_layer = Dense(1)(dense2)
    
    return output_layer

In [12]:
input_layer = Input((test2.shape[1], 1))
output_layer = build_model(input_layer, 512)

model = Model(input_layer, output_layer)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 180, 1)]     0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 180, 512)     2048        input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 180, 512)     786944      conv1d_7[0][0]                   
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 180, 512)     2048        conv1d_8[0][0]                   
_______________________________________________________________________________________

## test 취급액 예측

In [13]:
# model 생성
input_layer = Input((test.shape[1], 1))
output_layer = build_model(input_layer, 512)
model = Model(input_layer, output_layer)

# model 가중치 불러오기
model.load_weights('drive/Shared drives/빅콘테스트2020/weights/yj/functional_1.50-34.5923.hdf5')

# 주문량 예측
y_pred = model.predict(test2.values)

In [17]:
real_test = pd.read_csv("/content/drive/Shared drives/빅콘테스트2020/data/data/02_평가데이터/test.csv")

In [21]:
real_test

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
1,2020-06-01 6:20,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800",NaN
2,2020-06-01 6:40,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800",NaN
3,2020-06-01 7:00,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,"59,800",NaN
4,2020-06-01 7:20,20,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,"69,900",NaN
5,2020-06-01 7:40,20,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,"69,900",NaN
...,...,...,...,...,...,...,...,...
2887,2020-07-01 0:20,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-,NaN
2888,2020-07-01 0:40,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-,NaN
2889,2020-07-01 1:00,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,-,NaN
2890,2020-07-01 1:20,20,100261,200875,아놀드파마 티셔츠레깅스세트,의류,"69,900",NaN


In [20]:
real_test.columns = real_test.iloc[0]

real_test.drop([0], axis=0, inplace=True)

In [22]:
real_test['판매단가'] = ['0' if x == ' - ' else x for x in real_test['판매단가']]

In [25]:
real_test['판매단가'] = [x.replace(',','').strip() for x in real_test['판매단가']]

In [26]:
real_test['판매단가'] = real_test['판매단가'].astype('int')

real_test['취급액'] = y_pred.reshape(-1) * real_test['판매단가']
real_test

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
1,2020-06-01 6:20,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,2193714.49
2,2020-06-01 6:40,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,4858649.63
3,2020-06-01 7:00,20,100650,201971,잭필드 남성 반팔셔츠 4종,의류,59800,3040800.48
4,2020-06-01 7:20,20,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,13350956.53
5,2020-06-01 7:40,20,100445,202278,쿠미투니카 쿨 레이시 란쥬쉐이퍼&팬티,속옷,69900,20505365.95
...,...,...,...,...,...,...,...,...
2887,2020-07-01 0:20,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,0.00
2888,2020-07-01 0:40,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,0.00
2889,2020-07-01 1:00,20,100660,201989,쉴렉스 안마의자 렌탈서비스,무형,0,0.00
2890,2020-07-01 1:20,20,100261,200875,아놀드파마 티셔츠레깅스세트,의류,69900,2757700.91


# (*) 참고

## 같은 상품이어도 가격이 다르다
### * 높은 가격 이후에 가격이 낮아짐
### * 단지 안 팔려서 낮아진것 같지는 않아보임

In [ ]:
pd.set_option('display.max_rows', 1881)

b = pd.DataFrame(final.groupby(['상품명', '판매단가'])['방송일시'].count())

dic3 = {}

for i, j in b.index:
    if dic3.get(i) == None:
        dic3[i] = 1
    else:
        dic3[i] += 1

In [ ]:
name = []

for k, v in dic3.items():
    if v > 1:
        name.append(k)

In [ ]:
temp = final[final['상품명'] == name[25]][['방송일시', '판매단가', '취급액']]
sns.relplot(x="방송일시", y="취급액", hue='판매단가', data=temp, kind='line')

## 상품명 == 상품코드?
### * 데이터상으로는 특별한 이유를 찾기 힘들어보임
### * 결론은 둘이 같이 보면 안된다

In [ ]:
product = final['상품명'].value_counts().index
product

In [ ]:
code = final['상품코드'].value_counts().sort_index().index
code

In [ ]:
dic = {}

for i in tqdm(range(final.shape[0])):
    temp = final.iloc[i]
    dic[temp['상품코드']] = temp['상품명']

In [ ]:
dic2 = {}

for k, v in dic.items():
    if dic2.get(v) == None:
        dic2[v] = 1
    else:
        dic2[v] += 1

In [ ]:
for k, v in dic2.items():
    if v > 1:
        print(k, '->', v)